In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import requests
import json

In [2]:
# Read CSV file as Pandas Dataframe
person_df = pd.read_csv('Data/Person_FARS.csv', encoding='cp1252')

C:\Users\shrey\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Display the dataframe and filter
person_df.head(10)

pers_df = person_df.filter(['ST_CASE','MAKENAME', 'MAK_MOD', 'AGE', 'SEXNAME', 'DOANAME', 'PER_TYPNAME'], axis=1)
pers_df

,ST_CASE,MAKENAME,MAK_MOD,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,10001,Toyota,Toyota Camry,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
1,10001,Toyota,Toyota Camry,53,Male,Died at Scene,Passenger of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...
82215,560120,Hyundai,Hyundai Santa Fe,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82216,560120,Hyundai,Hyundai Santa Fe,22,Male,Not Applicable,Passenger of a Motor Vehicle In-Transport
82217,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82218,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,41,Female,Died at Scene,Passenger of a Motor Vehicle In-Transport


In [4]:
# Remove unnecessary rows
na_pers = pers_df[~pers_df.isin(["NOT APPLICABLE", "Not Reported", "Reported as Unknown", "Other", 999, 998]).any(axis=1)]
na_pers

,ST_CASE,MAKENAME,MAK_MOD,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,10001,Toyota,Toyota Camry,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
1,10001,Toyota,Toyota Camry,53,Male,Died at Scene,Passenger of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...
82215,560120,Hyundai,Hyundai Santa Fe,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82216,560120,Hyundai,Hyundai Santa Fe,22,Male,Not Applicable,Passenger of a Motor Vehicle In-Transport
82217,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82218,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,41,Female,Died at Scene,Passenger of a Motor Vehicle In-Transport


In [5]:
only_drivers = na_pers.loc[na_pers['PER_TYPNAME'] == "Driver of a Motor Vehicle In-Transport"]
only_drivers["PER_TYPNAME"].value_counts()
only_drivers.head()

,ST_CASE,MAKENAME,MAK_MOD,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,10001,Toyota,Toyota Camry,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
5,10003,Acura,Acura RL/RLX,22,Male,Not Applicable,Driver of a Motor Vehicle In-Transport


In [6]:
sc_pers_df = only_drivers.rename(columns={"ST_CASE": "Case_Number", "MAKENAME": "Vehicle_Manufacturer", "MAK_MOD": "Vehicle_Model",
                                             "SEXNAME": "Sex", "DOANAME": "DOA_Status", "PER_TYPNAME": "Passenger_Type"})

cleaned_pers_df = sc_pers_df.replace({'Not Applicable': 'No Fatality', 'Died at Scene': 'Fatal', 'nan': 'No Fatality', 'Died En Route': 'Fatal'})

cleaned_pers_df

,Case_Number,Vehicle_Manufacturer,Vehicle_Model,AGE,Sex,DOA_Status,Passenger_Type
0,10001,Toyota,Toyota Camry,34,Female,No Fatality,Driver of a Motor Vehicle In-Transport
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,No Fatality,Driver of a Motor Vehicle In-Transport
3,10002,BMW,BMW 3-series,42,Female,Fatal,Driver of a Motor Vehicle In-Transport
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,No Fatality,Driver of a Motor Vehicle In-Transport
5,10003,Acura,Acura RL/RLX,22,Male,No Fatality,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...
82213,560118,Peterbilt,Peterbilt Medium/Heavy - CBE,56,Male,No Fatality,Driver of a Motor Vehicle In-Transport
82214,560119,Harley-Davidson,Harley-Davidson 750cc or greater,52,Male,No Fatality,Driver of a Motor Vehicle In-Transport
82215,560120,Hyundai,Hyundai Santa Fe,31,Male,Fatal,Driver of a Motor Vehicle In-Transport
82217,560121,Dodge,Dodge Dart (2013 on. See model 001 for 1960-19...,42,Male,Fatal,Driver of a Motor Vehicle In-Transport


In [7]:
# Establish bins for ages

# bins: 1 = 6 - 19; 2 = 19 - 33; 3 = 33 - 46; 4 = 46 - 60; 5 = 60 - 73; 6 = 73 - 87; 7 = 87+

for row in cleaned_pers_df.iteritems():
    cleaned_pers_df['Age_Bin'] = pd.cut(cleaned_pers_df['AGE'],7, precision=0)
    cleaned_pers_df['Age_Label'] = pd.cut(cleaned_pers_df['AGE'],7, precision=0, labels=["1", "2", "3", "4", "5", "6", "7"])
    

In [8]:
cleaned_pers_df.head(50)

,Case_Number,Vehicle_Manufacturer,Vehicle_Model,AGE,Sex,DOA_Status,Passenger_Type,Age_Bin,Age_Label
0,10001,Toyota,Toyota Camry,34,Female,No Fatality,Driver of a Motor Vehicle In-Transport,"(33.0, 46.0]",3
2,10001,Freightliner,Freightliner Medium/Heavy - CBE,59,Male,No Fatality,Driver of a Motor Vehicle In-Transport,"(46.0, 60.0]",4
3,10002,BMW,BMW 3-series,42,Female,Fatal,Driver of a Motor Vehicle In-Transport,"(33.0, 46.0]",3
4,10002,Chevrolet,Chevrolet TrailBlazer (2003 on; for 2002 model...,54,Female,No Fatality,Driver of a Motor Vehicle In-Transport,"(46.0, 60.0]",4
5,10003,Acura,Acura RL/RLX,22,Male,No Fatality,Driver of a Motor Vehicle In-Transport,"(19.0, 33.0]",2
7,10003,International Harvester/Navistar,International Harvester/Navistar Medium/Heavy ...,22,Male,No Fatality,Driver of a Motor Vehicle In-Transport,"(19.0, 33.0]",2
8,10003,Ford,Ford Fusion,26,Female,No Fatality,Driver of a Motor Vehicle In-Transport,"(19.0, 33.0]",2
10,10004,Peterbilt,Peterbilt Medium/Heavy - CBE,41,Male,No Fatality,Driver of a Motor Vehicle In-Transport,"(33.0, 46.0]",3
11,10005,Toyota,Toyota Avalon,30,Male,Fatal,Driver of a Motor Vehicle In-Transport,"(19.0, 33.0]",2
12,10006,Freightliner,Freightliner Medium/Heavy - CBE,54,Male,Fatal,Driver of a Motor Vehicle In-Transport,"(46.0, 60.0]",4


In [8]:
cleaned_pers_df["Vehicle_Model"].value_counts()

Ford F-Series pickup                            2182
Chevrolet C, K, R, V-series pickup/Silverado    2037
Harley-Davidson 750cc or greater                1908
Dodge Ram Pickup                                1251
Freightliner Medium/Heavy - CBE                 1245
                                                ... 
Mack Unknown (MACK)                                1
Isuzu Medium/Heavy - COE, high entry               1
Dodge Medium/Heavy -Unknown engine location        1
Unknown Make Motor Home                            1
Chevrolet Lumina APV                               1
Name: Vehicle_Model, Length: 900, dtype: int64

In [9]:
cleaned_pers_df.drop(columns=["Age_Bin"], inplace=True)

In [10]:
# Writing Accidents final dataframe to a csv
cleaned_pers_df.to_csv(r"Output/person_final.csv", index=False, header=True)